# Welcome to ESS-DIVE's Jupyter Notebook for Sharing Datasets
This Jupyter Notebook will help **data contributors**: 
- Find out who currently can access their datasets and under which access levels
- Grant access to view, edit, or manage their datasets

Written By: Madison Burrus (LBNL)

---

**This notebook uses ESS-DIVE's test system, Sandbox, by default.** When you are ready to share your datasets on ESS-DIVE's main repository, change the default endpoint.

In [1]:
# This notebook requires Python 3.
# ===================================

from ipywidgets import widgets, interact
import json
import requests
import os

# 1. Setup ESS-DIVE API

Click on the link to login to ESS-DIVE, navigate to settings > authentication token and copy token: https://data-sandbox.ess-dive.lbl.gov/


**This token must be updated every 24hrs**

In [2]:
# Setup token
token = widgets.Text("",description="Token:") 
display(token)

Text(value='', description='Token:')

In [3]:
# Setup Dataset Identifier
print("Select a dataset on Sandbox that you have created to test out sharing: https://data-sandbox.ess-dive.lbl.gov")
dataset_identifier = input('Dataset Identifier Here: ')

Select a dataset on Sandbox that you have created to test out sharing: https://data-sandbox.ess-dive.lbl.gov
Dataset Identifier Here: ess-dive-c0841df396bf9f4-20231102T164859885


In [4]:
# Setup ESS-DIVE data package API
base = widgets.Text("https://api-sandbox.ess-dive.lbl.gov/",description="Default Endpoint:")
display(base)

Text(value='https://api-sandbox.ess-dive.lbl.gov/', description='Default Endpoint:')

**Always re-run the following cell when you update any of the above inputs**

In [5]:
# Re-run this cell whenever your inputs change

header_authorization =  f"bearer {token.value}"
endpoint = "packages"
permission_endpoint = "share"

# 2. Lookup Dataset Permissions

This uses the call `GET/ packages/{identifier}/share`. It will tell you who currently has access to your dataset. 

Detailed information on this call is available in the Dataset API technical documentation: [Lookup Dataset Permissions](https://api-sandbox.ess-dive.lbl.gov/#/Dataset%20Sharing/getDatasetPermissions)

In [6]:
get_permission_url = "{}{}/{}/{}".format(base.value,endpoint,dataset_identifier,permission_endpoint)

get_permission_response = requests.get(get_permission_url,
                                      headers={"Authorization":header_authorization})

if get_permission_response.status_code == 200:
    # Success
    response=get_permission_response.json()
    print("The following ORCiD accounts currently have access to this dataset:")
    display(response)
else:
    # There was an error
    print(get_permission_response.text)

The following ORCiD accounts currently have access to this dataset:


{'id': 'ess-dive-c0841df396bf9f4-20231102T164859885',
 'accessPolicy': {'manage': ['http://orcid.org/0000-0003-2296-4698'],
  'edit': [],
  'view': []}}

# 3. Save Current Dataset Permissions

In [7]:
# Define file name, directory name, and file paths
file_name = 'dataset-permissions.json'
new_dir = 'ESS-DIVE_Downloads'
parent_dir = os.getcwd()
path = os.path.join(parent_dir, new_dir)
file_path=os.path.join(path, file_name)

In [8]:
# attempt to make the new folder if it doesn't already exist
try: 
    os.mkdir(path)
    print("Directory '% s' created" % new_dir)
except OSError as error: 
    print(error)  

Directory 'ESS-DIVE_Downloads' created


In [12]:
# Save permissions file
try:    
    with open(file_path, 'w') as fp:
        json.dump(response, fp)
    print("Your dataset permission file has been saved!")
except:
    print("Save failed.")

Your dataset permission file has been saved!


# 4. Change Dataset Permissions & Share Dataset

This uses the call `PUT/packages/{identifier}/share`. **This call will overwrite your current dataset permissions.** 

Detailed information on this call is available in the Dataset API technical documentation: 
[Update Dataset Permissions](https://api-sandbox.ess-dive.lbl.gov/#/Dataset%20Sharing/updateDatasetPermissions)

### Define request body (required)
There are three possible access levels that you can grant. [Learn about data permissions on ESS-DIVE](https://docs.ess-dive.lbl.gov/manage-data/share-data-permissions).

Formatting requirements for the request body are explained in detail in the technical documentation under "Schemas" > "AccessPolicy".

In [10]:
# Enter user orcids in the format `"http://orcid.org/XXX-XXXX-XXXX-XXXX"` 

json_permission = {
    "manage": [],
    "edit": [],
    "view": ["http://orcid.org/0000-0002-0439-9173"]
}

### Send PUT Request

**Infomrational Warning!**  The people you share your dataset with must have logged into ESS-DIVE and/or Sandbox with the ORCID you provided. It is not possible to share a dataset with someone who has not logged into ESS-DIVE before.

In [11]:
put_permission_url = "{}{}/{}/{}".format(base.value,endpoint,dataset_identifier,permission_endpoint)

put_permission_response = requests.put(put_permission_url, 
                                       headers={'Authorization': header_authorization,
                                                'Content-Type': 'application/json'}, 
                                       data=json.dumps(json_permission))

if put_permission_response.status_code == 200:
    # Success
    response=put_permission_response.json()
    print(f"The following ORCiD accounts have been added to the dataset, {dataset_identifier}:")
    display(response)
else:
    # There was an error
    print(put_permission_response.text)

The following ORCiD accounts have been added to the dataset, ess-dive-c0841df396bf9f4-20231102T164859885:


{'detail': 'Access policy successfully updated.',
 'id': 'ess-dive-c0841df396bf9f4-20231102T164859885',
 'accessPolicy': {'manage': [],
  'edit': [],
  'view': ['http://orcid.org/0000-0002-0439-9173']}}